# Анализ рейтингов фильмов

In [2]:
import pandas as pd
dates = pd.read_csv('data/dates.csv')
movies = pd.read_csv('data/movies.csv')
ratings1 = pd.read_csv('data/ratings1.csv')
ratings2 = pd.read_csv('data/ratings2.csv')
ratings_movies = pd.read_csv('data/ratings_movies.csv')


In [3]:
rr = ratings_movies
rr.head()

,Unnamed: 0,userId,movieId,rating,date,title,genres
0,0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [35]:
rr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   Unnamed: 0  100836 non-null  int64         
 1   userId      100836 non-null  int64         
 2   movieId     100836 non-null  int64         
 3   rating      100836 non-null  float64       
 4   date        100836 non-null  datetime64[ns]
 5   title       100836 non-null  object        
 6   genres      100836 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 5.4+ MB


In [18]:
ratings_movies.date = pd.to_datetime(ratings_movies.date)
ratings_movies.year = ratings_movies.date.dt.year
ratings_movies.year.value_counts()[:3]

C:\Users\Александр\AppData\Local\Temp\ipykernel_3276\261240958.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  ratings_movies.year = ratings_movies.date.dt.year


2000    10061
2017     8198
2007     7114
Name: date, dtype: int64

Задание 8.1

Создайте в таблице новый признак year_release, который соответствует году выпуска фильма.
У скольких фильмов не указан год их выпуска?

In [5]:
def get_year(text):

    # print(text[-1])
    # print(text[-6])
    # print(text[-5:-1])

    if len(text) >= 6 and text[-6] == '(' and text[-1] == ')':
        str_year = text[-5:-1]
        return  int(str_year)
    else:
        return None

import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None


rr['release'] = rr.title.apply(get_year_release)
# y=get_year('Grumpier Old Men (1995)')
# print(rr)
rr.info()
100818-100836  
# rr.groupby(by='title       ')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  100836 non-null  int64  
 1   userId      100836 non-null  int64  
 2   movieId     100836 non-null  int64  
 3   rating      100836 non-null  float64
 4   date        100836 non-null  object 
 5   title       100836 non-null  object 
 6   genres      100836 non-null  object 
 7   release     100818 non-null  float64
dtypes: float64(2), int64(3), object(3)
memory usage: 6.2+ MB


-18

Задание 8.2

Какой фильм, выпущенный в 1999 году, получил наименьшую среднюю оценку зрителей?
В качестве ответа запишите название этого фильма без указания года его выпуска.

In [69]:
tmp = rr[rr.release==1999].groupby('title').agg('mean')
tmp[tmp.rating == tmp.rating.min()]

,Unnamed: 0,userId,movieId,rating,release
title,,,,,
Bloodsport: The Dark Kumite (1999),100797.0,610.0,145951.0,0.5,1999.0


Задание 8.3

Какое сочетание жанров фильмов (genres), выпущенных в 2010 году, получило наименьшую среднюю оценку (rating)?
Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).

In [70]:
tmp = rr[rr.release==2010].groupby('genres').agg('mean')
tmp[tmp.rating == tmp.rating.min()]

,Unnamed: 0,userId,movieId,rating,release
genres,,,,,
Action|Sci-Fi,30179.0,210.0,189547.0,1.0,2010.0


Задание 8.4

Какой пользователь (userId) посмотрел наибольшее количество различных (уникальных) жанров (genres) фильмов? В качестве ответа запишите идентификатор этого пользователя

In [88]:
# pd.pivot_table(rr, index='userId', columns='genres').agg('count')
tmp = rr.groupby('userId')['genres'].agg('nunique',set)
# tmp.info()
# tmp[
tmp[tmp==tmp.max()]
# ==tmp.userId.max()]



userId
599    524
Name: genres, dtype: int64

Задание 8.5

Найдите пользователя, который выставил наименьшее количество оценок, но его средняя оценка фильмам наибольшая.
В качестве ответа укажите идентификатор этого пользователя.

In [121]:
u_count = rr.groupby('userId')['rating'].agg('count')
u_count = u_count.rename('count_r')
u_rating_mean = rr.groupby('userId')['rating'].agg('mean')
pd.concat([u_count, u_rating_mean], axis=1).sort_values(by=['count_r','rating'], ascending=[True,False])[0:3]

,count_r,rating
userId,,
53,20,5.0
595,20,4.2


Задание 8.6

Найдите сочетание жанров (genres) за 2018 году, которое имеет наибольший средний рейтинг (среднее по столбцу rating), и при этом число выставленных ему оценок (количество значений в столбце rating) больше 10.
Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).

In [14]:
r2018 = rr[rr.release==2018]
s1 = r2018.groupby("genres")['rating'].agg('mean')
s2 = r2018.groupby("genres")['rating'].agg('count').rename('count_r')
df = pd.concat([s1, s2], axis=1)
df[df.count_r > 7].sort_values(by=['count_r','rating'], ascending=[True,False])[0:30]


,rating,count_r
genres,,
Comedy,3.312500,8
Action|Comedy|Sci-Fi,3.875000,12
Action|Adventure|Sci-Fi,3.928571,14


In [31]:
rr['year_rating'] = rr.date.dt.year

pivot = rr.pivot_table(
    values='rating',
    index='year_rating',
    columns='genres',
    # aggfunc='median',
    aggfunc='mean',
    fill_value=0
)

pivot['Comedy']
#  pivot.groupby('year_rating', axis=0).agg('max')

year_rating
1996    3.228571
1997    3.409091
1998    3.000000
1999    3.606061
2000    3.141291
2001    3.318408
2002    3.198556
2003    3.120066
2004    3.356877
2005    2.963325
2006    3.347534
2007    2.928187
2008    3.426667
2009    3.273292
2010    3.179825
2011    3.232877
2012    3.478477
2013    3.489474
2014    3.271429
2015    3.106183
2016    3.373431
2017    2.852668
2018    3.121296
Name: Comedy, dtype: float64

In [58]:

products = pd.read_csv('data/products.csv', sep=';')
orders = pd.read_csv('data/orders.csv', sep=';')

In [61]:
orders = orders.rename(columns= {"ID товара": 'Product_ID'}) 
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Дата создания  18 non-null     object
 1   Order ID       18 non-null     int64 
 2   ID Покупателя  18 non-null     int64 
 3   Статус         18 non-null     object
 4   Оплачен        18 non-null     object
 5   Отменен        18 non-null     object
 6   Отгружен       18 non-null     object
 7   Product_ID     18 non-null     int64 
 8   Количество     18 non-null     int64 
dtypes: int64(4), object(5)
memory usage: 1.4+ KB


In [48]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Product_ID  14 non-null     int64 
 1   Name        14 non-null     object
 2   Price       14 non-null     int64 
 3   CURRENCY    14 non-null     object
dtypes: int64(2), object(2)
memory usage: 576.0+ bytes


In [65]:
orders_products=orders.merge(
    products, 
    how='left',
    left_on='Product_ID',
    right_on='Product_ID'
)

orders_products['summ'] = orders_products.Price*orders_products.Количество
orders_products[orders_products.Оплачен == 'Да'].groupby('ID Покупателя')['summ'].sum()

ID Покупателя
1        0.0
5    13043.0
7    17096.0
8     1344.0
Name: summ, dtype: float64